## Stage 2: Feature Engineering and Operational EDA

### Objective
Develop robust features that capture operational causality (historical risk,
external factors) and conduct targeted EDA to build a strong narrative for
stakeholders. This stage implements solutions to key feedback points:
1. Historical/Causal Features.
2. Robust Imputation (Indicator variables for missing data).
3. Clear EDA Narrative (Focusing on Delay Rate for intervention points).

### 1. Configuration and Library Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Setting configurations
sns.set_style("whitegrid")
pd.set_option('display.max_columns', 100)
%matplotlib inline